In [0]:
import os
!rm -rf *
!rm -rf .kaggle/
!git clone https://github.com/bkahn-github/data-science-bowl.git
!pip install -r data-science-bowl/requirements.txt
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip3 install Pillow
!mkdir ~/.kaggle
f = open(".kaggle/kaggle.json", "w")
f.write('{"username":"bkkaggle","key":"bf4e62680116284087ace0484990f87a"}')
f.close()
!kaggle competitions download -c data-science-bowl-2018
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage1_train.zip -d ~/.kaggle/competitions/data-science-bowl-2018/train
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage1_test.zip -d ~/.kaggle/competitions/data-science-bowl-2018/test
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage1_train_labels.csv.zip -d ~/.kaggle/competitions/data-science-bowl-2018/labels
os.chdir('data-science-bowl/kerasUnet')
!git checkout new-keras-unet

In [0]:
os.chdir('../../')
!rm -rf data-science-bowl/
!git clone https://github.com/bkahn-github/data-science-bowl.git
os.chdir('data-science-bowl/kerasUnet')
!git checkout new-keras-unet

In [2]:
import os
import sys
import random
import warnings

import PIL
import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator

import imageio
import matplotlib.pyplot as plt
from skimage.transform import resize
from tqdm import tqdm, tqdm_notebook

from load_data import load_data, load_test_data, load_test_image_sizes
from metrics import dice_loss, iou
from model import model
from process_data import encode, upsample
from submit import submit

%matplotlib inline

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

Using TensorFlow backend.


In [3]:
x_train, y_train, x_val, y_val = load_data(train_val_split=0.1)
x_test = load_test_data()
x_test_sizes = load_test_image_sizes()

100%|██████████| 65/65 [00:00<00:00, 138.53it/s]


In [4]:
unet = model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 128, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
unet.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=4, epochs=25)

Train on 603 samples, validate on 67 samples
Epoch 1/25
603/603 [==============================] - 17s 28ms/step - loss: 0.3481 - keras_iou: 0.4198 - val_loss: 0.2477 - val_keras_iou: 0.4546
Epoch 2/25
603/603 [==============================] - 14s 23ms/step - loss: 0.1454 - keras_iou: 0.5136 - val_loss: 0.1310 - val_keras_iou: 0.5714
Epoch 3/25
320/603 [==============>...............] - ETA: 6s - loss: 0.1128 - keras_iou: 0.5963

603/603 [==============================] - 14s 23ms/step - loss: 0.1101 - keras_iou: 0.6113 - val_loss: 0.1091 - val_keras_iou: 0.6428
Epoch 4/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0984 - keras_iou: 0.6668 - val_loss: 0.0970 - val_keras_iou: 0.6863
Epoch 5/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0924 - keras_iou: 0.7016 - val_loss: 0.0910 - val_keras_iou: 0.7154
Epoch 6/25
 12/603 [..............................] - ETA: 12s - loss: 0.1155 - keras_iou: 0.7166

603/603 [==============================] - 14s 23ms/step - loss: 0.0860 - keras_iou: 0.7262 - val_loss: 0.0915 - val_keras_iou: 0.7360
Epoch 7/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0842 - keras_iou: 0.7446 - val_loss: 0.0944 - val_keras_iou: 0.7512
Epoch 8/25
552/603 [==========================>...] - ETA: 1s - loss: 0.0827 - keras_iou: 0.7566

603/603 [==============================] - 14s 23ms/step - loss: 0.0813 - keras_iou: 0.7571 - val_loss: 0.0832 - val_keras_iou: 0.7630
Epoch 9/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0784 - keras_iou: 0.7678 - val_loss: 0.0898 - val_keras_iou: 0.7728
Epoch 10/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0834 - keras_iou: 0.7762 - val_loss: 0.0856 - val_keras_iou: 0.7800
Epoch 11/25
 48/603 [=>............................] - ETA: 12s - loss: 0.0753 - keras_iou: 0.7805

603/603 [==============================] - 14s 23ms/step - loss: 0.0768 - keras_iou: 0.7835 - val_loss: 0.0852 - val_keras_iou: 0.7866
Epoch 12/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0749 - keras_iou: 0.7896 - val_loss: 0.0769 - val_keras_iou: 0.7924
Epoch 13/25
556/603 [==========================>...] - ETA: 1s - loss: 0.0772 - keras_iou: 0.7946

603/603 [==============================] - 14s 23ms/step - loss: 0.0767 - keras_iou: 0.7948 - val_loss: 0.0820 - val_keras_iou: 0.7973
Epoch 14/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0756 - keras_iou: 0.7992 - val_loss: 0.0808 - val_keras_iou: 0.8016
Epoch 15/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0737 - keras_iou: 0.8035 - val_loss: 0.0803 - val_keras_iou: 0.8053
Epoch 16/25
 48/603 [=>............................] - ETA: 12s - loss: 0.0785 - keras_iou: 0.8056

603/603 [==============================] - 14s 23ms/step - loss: 0.0715 - keras_iou: 0.8071 - val_loss: 0.0772 - val_keras_iou: 0.8088
Epoch 17/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0710 - keras_iou: 0.8105 - val_loss: 0.0754 - val_keras_iou: 0.8121
Epoch 18/25
556/603 [==========================>...] - ETA: 1s - loss: 0.0699 - keras_iou: 0.8134

603/603 [==============================] - 14s 23ms/step - loss: 0.0691 - keras_iou: 0.8135 - val_loss: 0.0740 - val_keras_iou: 0.8149
Epoch 19/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0681 - keras_iou: 0.8163 - val_loss: 0.0782 - val_keras_iou: 0.8177
Epoch 20/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0767 - keras_iou: 0.8186 - val_loss: 0.0783 - val_keras_iou: 0.8197
Epoch 21/25
 48/603 [=>............................] - ETA: 12s - loss: 0.0669 - keras_iou: 0.8198

603/603 [==============================] - 14s 23ms/step - loss: 0.0714 - keras_iou: 0.8207 - val_loss: 0.0744 - val_keras_iou: 0.8216
Epoch 22/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0681 - keras_iou: 0.8226 - val_loss: 0.0759 - val_keras_iou: 0.8237
Epoch 23/25
556/603 [==========================>...] - ETA: 1s - loss: 0.0690 - keras_iou: 0.8246

603/603 [==============================] - 14s 23ms/step - loss: 0.0681 - keras_iou: 0.8246 - val_loss: 0.0708 - val_keras_iou: 0.8255
Epoch 24/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0673 - keras_iou: 0.8266 - val_loss: 0.0726 - val_keras_iou: 0.8274
Epoch 25/25
603/603 [==============================] - 14s 23ms/step - loss: 0.0669 - keras_iou: 0.8282 - val_loss: 0.0704 - val_keras_iou: 0.8291


In [0]:
preds = unet.predict(x_test)

In [0]:
test_path = '../../.kaggle/competitions/data-science-bowl-2018/test/'
test_ids = next(os.walk(test_path))[1]

preds_upsampled = upsample(preds)

In [0]:
rles, new_test_ids = encode(preds_upsampled, test_ids)

In [0]:
submit(new_test_ids, rles)

In [0]:
pd.read_csv('./submission.csv')

In [0]:
from google.colab import files

files.download('submission.csv')